# CP01 – Data Science e ML: Consumo de Energia (UCI)
Notebook completo (tarefas 1–20) • Curso: **IA – André Tritiack**

**Prazo:** 2 de setembro de 2025 – 23:59

> Dataset: *Individual household electric power consumption* (UCI)
> Link oficial: https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

**Instruções:**
- Rode este notebook localmente com Python 3.10+.
- Ele fará o *download* e a extração automáticos do dataset, caso necessário.
- Dependências: pandas, numpy, matplotlib, scikit-learn, statsmodels, requests, zipfile.
- Se preferir, coloque manualmente o arquivo `household_power_consumption.txt` na pasta `data/`.


In [ ]:

# === Setup: imports, diretórios e (opcional) download ===
import os
import io
import zipfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings("ignore")

DATA_DIR = "data"
os.makedirs(DATA_DIR, exist_ok=True)

TXT_PATH = os.path.join(DATA_DIR, "household_power_consumption.txt")
ZIP_PATH = os.path.join(DATA_DIR, "household_power_consumption.zip")

# URLs conhecidas do UCI
CANDIDATE_URLS = [
    # zip com o .txt
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.txt",
]

def try_download():
    if os.path.exists(TXT_PATH):
        print("Arquivo .txt já encontrado em", TXT_PATH)
        return
    
    for url in CANDIDATE_URLS:
        try:
            print("Tentando baixar:", url)
            resp = requests.get(url, timeout=60)
            resp.raise_for_status()
            content_type = resp.headers.get("Content-Type", "")
            # Se for zip, extrai
            if url.endswith(".zip") or "zip" in content_type.lower():
                with open(ZIP_PATH, "wb") as f:
                    f.write(resp.content)
                with zipfile.ZipFile(ZIP_PATH, "r") as z:
                    z.extractall(DATA_DIR)
                if os.path.exists(TXT_PATH):
                    print("Download e extração concluídos.")
                    return
            else:
                # é o .txt direto
                with open(TXT_PATH, "wb") as f:
                    f.write(resp.content)
                print("Download do .txt concluído.")
                return
        except Exception as e:
            print("Falha ao baixar de", url, "->", str(e))
    
    print("\nNão foi possível baixar automaticamente. "
          "Faça o download manual e coloque 'household_power_consumption.txt' na pasta 'data/'.")

try_download()


In [ ]:

# === Carregamento do dataset (1) ===
# O arquivo possui separador ';' e valores ausentes como '?'. Datas estão em 'Date' (dd/mm/yyyy) e 'Time' (HH:MM:SS).
parse_dates = {"DateTime": ["Date", "Time"]}

df = pd.read_csv(
    TXT_PATH,
    sep=";",
    na_values="?",
    low_memory=False
)

# Cria DateTime combinando Date e Time para facilitar análises temporais
df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%d/%m/%Y %H:%M:%S", errors="coerce")

# Converte as colunas numéricas relevantes
num_cols = [
    "Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity",
    "Sub_metering_1", "Sub_metering_2", "Sub_metering_3"
]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Ordem por tempo e index
df = df.sort_values("DateTime").reset_index(drop=True)

# Visualiza 10 primeiras linhas (1)
df.head(10)



## (2) Diferença entre *Global_active_power* e *Global_reactive_power*

- **Global_active_power** (kW): potência **ativa** instantânea consumida pela residência. É a parcela efetivamente convertida em trabalho/energia útil (o que vira kWh na conta).
- **Global_reactive_power** (kVAR): potência **reativa** associada a campos elétricos/magnéticos em cargas indutivas/capacitivas. Não realiza trabalho útil, mas circula entre carga e rede, afetando fator de potência.


In [ ]:

# (3) Valores ausentes: contagem por coluna
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_counts


In [ ]:

# (4) Converte 'Date' em datetime (já temos 'DateTime', mas manteremos 'Date_dt') e cria 'Weekday'
df["Date_dt"] = pd.to_datetime(df["Date"], format="%d/%m/%Y", errors="coerce")
df["Weekday"] = df["Date_dt"].dt.day_name(locale="en_US")  # nomes em inglês para compatibilidade
df[["Date", "Date_dt", "Weekday"]].head()


In [ ]:

# (5) Filtrar registros de 2007 e calcular média de consumo diário de Global_active_power
df_2007 = df[df["Date_dt"].dt.year == 2007].copy()
daily_mean_2007 = df_2007.groupby(df_2007["Date_dt"])["Global_active_power"].mean()
daily_mean_2007.describe(), daily_mean_2007.head()


In [ ]:

# (6) Gráfico de linha para a variação de Global_active_power em um único dia escolhido
# Escolha do dia: 2007-01-15 (ajuste se desejar outro dia)
one_day = pd.to_datetime("2007-01-15")
mask = (df["Date_dt"] == one_day)
day_series = df.loc[mask, ["DateTime", "Global_active_power"]].dropna()

plt.figure()
plt.plot(day_series["DateTime"], day_series["Global_active_power"])
plt.title("Global_active_power em 2007-01-15")
plt.xlabel("Hora")
plt.ylabel("kW")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# (7) Histograma de Voltage
plt.figure()
df["Voltage"].dropna().hist(bins=60)
plt.title("Distribuição de Voltage")
plt.xlabel("Volt")
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()

# Observação (em Markdown abaixo)



**Observação:** Em geral, a distribuição de `Voltage` tende a ser relativamente concentrada em torno de valores próximos a ~240 V, 
com certa dispersão e possíveis caudas devido a variações momentâneas na rede.


In [ ]:

# (8) Consumo médio por mês (todo o período)
df["YearMonth"] = df["Date_dt"].dt.to_period("M")
monthly_mean = df.groupby("YearMonth")["Global_active_power"].mean().to_frame("mean_Global_active_power")
monthly_mean.head(), monthly_mean.describe()


In [ ]:

# (9) Dia com maior consumo de energia ativa global
# Aproximação de energia diária (kWh) = soma de kW * (1/60) h, dado amostragem por minuto
daily_kwh = df.groupby("Date_dt")["Global_active_power"].sum() / 60.0
max_day = daily_kwh.idxmax()
max_value = daily_kwh.max()
max_day, float(max_value)


In [ ]:

# (10) Consumo médio em dias de semana vs finais de semana
df["IsWeekend"] = df["Date_dt"].dt.dayofweek >= 5  # 5,6 => sábado, domingo
weekday_vs_weekend = df.groupby("IsWeekend")["Global_active_power"].mean().rename({False:"Semana", True:"Fim de semana"})
weekday_vs_weekend


In [ ]:

# (11) Correlação entre variáveis selecionadas
corr_vars = ["Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity"]
corr_matrix = df[corr_vars].corr()
corr_matrix


In [ ]:

# (12) Nova variável: Total_Sub_metering
df["Total_Sub_metering"] = df["Sub_metering_1"].fillna(0) + df["Sub_metering_2"].fillna(0) + df["Sub_metering_3"].fillna(0)
df[["Sub_metering_1","Sub_metering_2","Sub_metering_3","Total_Sub_metering"]].head()


In [ ]:

# (13) Algum mês com Total_Sub_metering (média mensal) > média mensal de Global_active_power?
# Nota: Unidades não são diretamente comparáveis (Wh vs kW instantâneo), mas seguimos o enunciado.
monthly = df.groupby("YearMonth").agg(
    mean_GAP=("Global_active_power", "mean"),
    mean_TSM=("Total_Sub_metering", "mean")
)
check = monthly[monthly["mean_TSM"] > monthly["mean_GAP"]]
monthly.head(), check


In [ ]:

# (14) Série temporal de Voltage para 2008
df_2008 = df[df["Date_dt"].dt.year == 2008].copy()
ts_2008 = df_2008.set_index("DateTime")["Voltage"].dropna()

plt.figure()
ts_2008.plot()
plt.title("Voltage – Ano de 2008")
plt.xlabel("Data")
plt.ylabel("Volt")
plt.tight_layout()
plt.show()


In [ ]:

# (15) Verão (Jun–Ago) vs Inverno (Dez–Fev) no Hemisfério Norte
df["Month"] = df["Date_dt"].dt.month
summer_mask = df["Month"].isin([6,7,8])
winter_mask = df["Month"].isin([12,1,2])

summer_mean = df.loc[summer_mask, "Global_active_power"].mean()
winter_mean = df.loc[winter_mask, "Global_active_power"].mean()
{"summer_mean": float(summer_mean), "winter_mean": float(winter_mean)}


In [ ]:

# (16) Amostragem aleatória de 1% e comparação de distribuição
sample_1pct = df.sample(frac=0.01, random_state=42)
plt.figure()
df["Global_active_power"].dropna().plot(kind="hist", bins=60, density=True, alpha=0.5)
sample_1pct["Global_active_power"].dropna().plot(kind="hist", bins=60, density=True, alpha=0.5)
plt.title("Distribuição de Global_active_power: Base Completa vs Amostra 1%")
plt.xlabel("kW")
plt.ylabel("Densidade")
plt.legend(["Completa", "Amostra 1%"])
plt.tight_layout()
plt.show()


In [ ]:

# (17) Normalização Min-Max
scale_cols = ["Global_active_power","Global_reactive_power","Voltage","Global_intensity","Total_Sub_metering"]
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[scale_cols].fillna(df[scale_cols].median()))
df_scaled = pd.DataFrame(scaled, columns=[c+"_scaled" for c in scale_cols])
df_norm = pd.concat([df, df_scaled], axis=1)
df_norm[[c+"_scaled" for c in scale_cols]].describe()


In [ ]:

# (18) K-means para segmentar dias (k=3) com base em métricas diárias
daily = df.groupby("Date_dt").agg(
    GAP_mean=("Global_active_power","mean"),
    GAP_max=("Global_active_power","max"),
    GRP_mean=("Global_reactive_power","mean"),
    Volt_mean=("Voltage","mean"),
    GI_mean=("Global_intensity","mean"),
    TSM_mean=("Total_Sub_metering","mean")
).dropna()

X = daily.copy()
scaler_days = MinMaxScaler()
X_scaled = scaler_days.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = kmeans.fit_predict(X_scaled)
daily["Cluster"] = labels

cluster_profile = daily.groupby("Cluster").mean()
cluster_counts = daily["Cluster"].value_counts().sort_index()

cluster_counts, cluster_profile


In [ ]:

# (19) Decomposição de série temporal (6 meses) – usando média diária (mais estável)
from statsmodels.tsa.seasonal import seasonal_decompose

# Seleciona 6 meses contínuos (ex.: 2007-01-01 a 2007-06-30)
mask_6m = (df["Date_dt"] >= pd.to_datetime("2007-01-01")) & (df["Date_dt"] <= pd.to_datetime("2007-06-30"))
df_6m = df.loc[mask_6m].copy()
daily_gap = df_6m.set_index("Date_dt")["Global_active_power"].resample("D").mean().dropna()

# Decomposição aditiva com sazonalidade semanal (period=7)
result = seasonal_decompose(daily_gap, model="additive", period=7)

plt.figure(); result.observed.plot(title="Observado"); plt.tight_layout(); plt.show()
plt.figure(); result.trend.plot(title="Tendência"); plt.tight_layout(); plt.show()
plt.figure(); result.seasonal.plot(title="Sazonalidade (periodo=7)"); plt.tight_layout(); plt.show()
plt.figure(); result.resid.plot(title="Resíduo"); plt.tight_layout(); plt.show()


In [ ]:

# (20) Regressão linear simples: prever Global_active_power a partir de Global_intensity
data_lr = df[["Global_active_power","Global_intensity"]].dropna().copy()
X = data_lr[["Global_intensity"]].values
y = data_lr["Global_active_power"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("Coeficiente (inclinação):", lr.coef_[0])
print("Intercepto:", lr.intercept_)
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Dispersão + reta ajustada (amostra para leveza visual)
plot_sample = min(5000, len(X_test))
idx = np.random.RandomState(42).choice(len(X_test), size=plot_sample, replace=False)

plt.figure()
plt.scatter(X_test[idx], y_test[idx], alpha=0.3, s=10)
# reta
x_line = np.linspace(X_test.min(), X_test.max(), 200).reshape(-1,1)
y_line = lr.predict(x_line)
plt.plot(x_line, y_line)
plt.title("Regressão: Global_active_power ~ Global_intensity")
plt.xlabel("Global_intensity")
plt.ylabel("Global_active_power")
plt.tight_layout()
plt.show()
